# CUDA

이 책 전체에서 우리는 대부분 PyTorch 또는 fastai 및 Hugging Face 변환기와 같이 그 위에 구축된 도구를 사용했습니다. 이 책에서 PyTorch를 처음 소개했을 때 우리는 PyTorch를 선형 대수 지식을 사용하여 "처음부터" 아키텍처를 구축하고 훈련 루프를 작성하는 저수준 프레임워크로 제시했습니다.

그러나 PyTorch는 기계 학습에서 다루는 가장 낮은 수준의 추상화가 아닐 수 있습니다.

PyTorch 자체는 CUDA 언어가 확장된 C++로 작성되었습니다. CUDA는 Nvidia GPU용 코드를 작성할 수 있는 "프로그래밍 모델"이라고 자칭합니다. C++ 코드를 작성할 때 GPU에서 작업의 일부를 수행하는 "CUDA 커널"이라는 특정 기능을 포함합니다.

> 그 포켓몬은 누구입니까? CUDA 커널

> 커널은 GPU(그래픽 처리 장치), FPGA(필드 프로그래밍 가능 게이트 어레이) 및 ASIC(응용 프로그램별 통합 회로)과 같은 특수 가속기 하드웨어용으로 컴파일되고 실행되도록 설계된 기능입니다. 일반적으로 하드웨어 아키텍처에 매우 익숙한 엔지니어가 작성하며 행렬 곱셈 또는 컨벌루션과 같은 단일 작업을 매우 잘 수행하도록 광범위하게 조정됩니다. CUDA 커널은 CUDA-Nvidia의 GPU 및 가속기를 사용하는 장치에서 실행되는 커널입니다.

PyTorch 및 기타 많은 딥 러닝 프레임워크는 소수의 CUDA 커널을 사용하여 백엔드를 구현한 다음 Python과 같은 언어에 대한 상위 수준 인터페이스를 구축합니다. 이를 통해 전문가가 메모리, 포인터, 스레드 등에 대해 생각할 필요 없이 수년 동안 최적화한 특수 하드웨어에서 초고속 수동 조정 코드를 실행할 수 있습니다.

AMS의 ROCm, SYCL(Khronos Group의 오픈 소스 대안), 구석구석 AI 하드웨어 스타트업이 등장하는 등 유사한 플랫폼이 많이 있습니다.

그러나 CUDA는 현재 사용 가능한 가장 성숙하고 잘 개발된 GPU 프로그래밍 인터페이스입니다. 사실, 이것이 우리 모두가 Nvidia의 GPU를 사용하도록 강요받는 이유입니다. Nvidia의 소프트웨어 스택은 다른 모든 것보다 훨씬 우수하여 그 위에 PyTorch와 같은 라이브러리를 더 쉽게 개발할 수 있습니다.

대역폭이 없는 한 커널 수준의 개선 사항을 찾는 것이 항상 좋은 생각은 아닙니다. 기존 도구와 기술을 사용하여 NLP 애플리케이션을 배포하는 데 중점을 둔 경우 수행해야 할 작업 목록에서 아마도 매우 낮을 것입니다.

하지만...오늘날 딥 러닝을 지원하는 인프라의 중요한 구성 요소가 어떻게 작동하는지 이해하는 것은 유용하며 확실히 흥미롭고 재미있습니다. CUDA의 아이디어 중 일부에 대한 이해는 딥 러닝 프레임워크에서 모호한 오류를 디버깅하는 데 도움이 될 수 있으며 하드웨어에 대해 더 많은 정보에 입각한 구매 결정을 내리는 데 도움이 될 수 있습니다.

## Threads and Thread Blocks

CUDA의 기본 요소는 스레드입니다. 스레드는 단일 계산 실행 단위를 나타냅니다. 단일 스레드에서 실행되는 모든 명령어는 순차적으로 실행됩니다. 대규모 병렬 처리를 위해 CUDA 장치에는 일반적으로 모두 독립적으로 실행되는 많은 스레드가 있습니다.

결정적으로 스레드 간의 통신은 어렵기 때문에(일반 CPU에서도) 가능한 한 이를 피하려고 합니다. 이것이 믿기지 않는다면 파인애플이 피자에 속하는지 아닌지에 대해 100명의 사람들이 동의하도록 하십시오. 어렵기 때문에 CUDA는 문제를 상당 부분 회피하려고 시도하며 당혹스러울 정도로 병렬적인 문제에 훨씬 더 적합합니다.

> 예, "부끄럽게 병렬"은 몇 가지 상황에서 듣게 될 다소 널리 사용되는 기술 용어입니다. 일반적으로 해결하려는 문제가 각 항목에 의존하지 않는 여러 개의 작은 작업으로 구성되어 있음을 의미합니다. 이는 하이퍼파라미터 세트, 훈련 배치의 샘플, 심지어 변환기 시퀀스의 토큰 전체에 걸쳐 자연스러운 병렬 처리가 있는 딥 러닝에서 사실입니다.

CUDA의 스레드는 블록이라고 하는 것으로 배열되며, 블록 자체는 '그리드'로 배열됩니다.

## Writing CUDA Kernels

## CUDA in Practice

CUDA 커널 작성, 프로파일링 및 코드 조정은 재미있을 수 있지만 CUDA의 이점을 추출하기 위해 항상 이 추상화 수준에서 작업할 필요는 없습니다. 보여드린 예제는 현재 실제 세계에 배포되는 CUDA 코드보다 훨씬 간단합니다.

Python에서는 행렬 곱셈을 수행할 때 문서를 찾습니다. a.matmul(b), matmul(a,b) 및 a@b와 같은 몇 가지 구문 변형이 있을 수 있지만 그게 전부입니다. 우리는 일반적으로 이러한 방법에 대해 너무 많이 생각하지 않습니다.

CUDA는 완전히 다른 차원의 존재입니다. 어떤 시나리오에서 어떤 커널을 호출할지 결정하기 위한 복잡한 휴리스틱을 사용하여 경쟁하는 여러 행렬 곱셈 알고리즘이 있습니다. 사용되는 martix 곱셈의 구현은 행렬의 모양, 메모리 대역폭 및 기타 하드웨어 관련 세부 정보에 따라 크게 달라질 수 있습니다.

고맙게도 범용 GPU 코드를 위한 약간 더 나은 추상화 계층인 CUDA 라이브러리가 있습니다. 여기에는 CuFFT, cuDNN, cuSPARE 등이 포함됩니다. CUDA 라이브러리에는 컨볼루션, 푸리에 변환, 행렬 곱셈 등과 같이 GPU에서 실행할 수 있는 가장 일반적인 알고리즘의 고도로 최적화된 구현이 포함되어 있습니다.

또한 Torch::Tensor와 같은 더 높은 수준의 프리미티브를 제공하는 PyTorch C++ 라이브러리인 libtorch도 있습니다. PyTorch C++ 코드는 Python의 PyTorch 코드와 놀라울 정도로 유사합니다. 다음은 공식 가이드(https://oreil.ly/IEdxH)에서 PyTorch 문서에서 LLTM(Long Log-Term Memory)이라고 하는 사용자 지정 확장에 대한 계층의 예입니다.:

```
#include <vector>
std::vector<at::Tensor> lltm_forward(
    torch::Tensor input,
    torch::Tensor weights,
    torch::Tensor bias,
    torch::Tensor old_h,
    torch::Tensor old_cell
) {
    auto X = torch::cat({old_h, input}, /*dim=*/1);

    auto gate_weights = torch::addmm(bias, X, weights.transpose(0,1));
    auto gates = gate_weights.chunk(3, /*dim=*/1);

    auto input_gate = torch::sigmoid(gates[0]);
    auto output_gate = torch::sigmoid(gates[1]);
    auto candidate_cell = torch::elu(gates[2], /*alpha=*/1.0);

    auto new_cell = old_cell + candidate_cell * input_gate;
    auto new_h = torch::tanh(new_cell) * output_gate;

    return  {
        new_h,
        new_cell,
        input_gate,
        output_gate,
        candidate_cell,
        X,
        gate_weights
    };
}
```
이것은 여전히 ​​CUDA에서 수행할 포인터 조작보다 훨씬 더 높은 수준이지만, 새로운 사용자 지정 계층을 구현해야 하고 Python 코드를 짜맞추면 상당한 성능 저하가 발생하는 경우 실제로 매우 유용할 수 있습니다. 단순히 libtorch로 레이어를 작성하고, Python에 연결하고, 대신 이를 사용하는 행위는 눈에 띄는 속도 향상을 가져올 수 있으며, 이는 확실히 시간을 투자할 가치가 있는 최적화입니다.

실제로 저수준 GPU 코드를 작성하기 위한 첫 번째 단계를 밟고 싶지만 공유 메모리에서 반정밀도 푸리에 변환에 가장 효율적인 액세스 패턴이 무엇인지 알아내느라 귀중한 시간을 낭비하고 싶지 않다면 CUDA 라이브러리와 libtorch는 다음 NLP 생성을 만들 때 사용할 수 있는 훌륭한 도구입니다.